# Random Forest Regressor

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
dataset = pd.read_csv('bases/50_Startups.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


### TO-DO 1

Remova as colunas Administration e State do dataset

In [8]:
#resposta
dataset = dataset.drop(columns = ['Administration', 'State'])
dataset.head()

,R&D Spend,Marketing Spend,Profit
0,165349.20,471784.10,192261.83
1,162597.70,443898.53,191792.06
2,153441.51,407934.54,191050.39
3,144372.41,383199.62,182901.99
4,142107.34,366168.42,166187.94


### TO-DO 2
Separe o dataset em vetor de características e variável meta

In [9]:
#resposta
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

assert type(X) == np.ndarray, "É esperado que X seja um numpy.ndarray"
assert type(y) == np.ndarray, "É esperado que y seja um numpy.ndarray"

### TO-DO 3
Treine um modelo de regressão usando Regression Tree e outro usando Random Forest Regressor

In [10]:
#resposta
dt_regressor = DecisionTreeRegressor(random_state = 0)
dt_regressor.fit(X, y) 

rf_regressor = RandomForestRegressor()
rf_regressor.fit(X,y)

RandomForestRegressor()

### TO-DO 4
Suponha que a empresa tenha um orçamento de R$ 500.000,00.

Eles estão considerando três opções:

> R&D: 300.000,00 ; Marketing: 200.000,00

> R&D: 200.000,00 ; Marketing: 300.000,00

> R&D: 100.000,00 ; Marketing: 400.000,00

Qual opção irá, a partir do modelo treinado, retornar o maior lucro? Faça um para cada modelo

In [11]:
# resposta
orcamentos = [[300000, 200000], [200000,300000], [100000,400000]]

for orcamento in orcamentos:
    y_pred_dt = dt_regressor.predict([orcamento])
    y_pred_rf = rf_regressor.predict([orcamento])
    print("Orçamento:", orcamento,
          "\nLucro Predito - Decision Tree:", y_pred_dt)
    print("Orçamento:", orcamento,
          "\nLucro Predito - Randomm Forest:", y_pred_rf)
    print()

Orçamento: [300000, 200000] 
Lucro Predito - Decision Tree: [166187.94]
Orçamento: [300000, 200000] 
Lucro Predito - Randomm Forest: [171488.4501]

Orçamento: [200000, 300000] 
Lucro Predito - Decision Tree: [166187.94]
Orçamento: [200000, 300000] 
Lucro Predito - Randomm Forest: [170911.9301]

Orçamento: [100000, 400000] 
Lucro Predito - Decision Tree: [125370.37]
Orçamento: [100000, 400000] 
Lucro Predito - Randomm Forest: [149004.3059]



# Random Forest Classifier

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
#fazendo o download dos dados direto dos datasets presente na scikit-learn
data = datasets.load_breast_cancer()
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [5]:
#variável meta (target) -> aquilos que estamos tentando prever
#no caso: cancer benigno ou maligno
data.target_names

array(['malignant', 'benign'], dtype='<U9')

In [6]:
#linhas x colunas
data.data.shape

(569, 30)

In [7]:
#visualizando as 5 primeiras amostras
data.data[0:5]

array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
        8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
        3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
        1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
        1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e

## TODO 1
> Separe o dataset em conjunto de treino e teste, sendo 70% para treino e 30% para teste

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.3, random_state=0) 

## TODO 2
> Treine uma árvore de classificacao nesse conjunto de dados

> faca a predicao do conjunto de teste

> obtenha a Acurácia, Precision, Recall

In [13]:
clf = DecisionTreeClassifier(max_depth=2, random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2, random_state=42)

In [14]:
y_pred = clf.predict(X_test)

In [15]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.9473684210526315
Precision: 0.9714285714285714
Recall: 0.9444444444444444


## TODO 3
> Treine uma Random Forest Classifier nesse conjunto de dados

> faça a predição do conjunto de teste

> obtenha a Acurácia, Precision, Recall

In [16]:
rf = RandomForestClassifier(max_depth=2, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=42)

In [17]:
y_pred_rf = rf.predict(X_test)

In [18]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_rf))
print("Precision:",metrics.precision_score(y_test, y_pred_rf))
print("Recall:",metrics.recall_score(y_test, y_pred_rf))

Accuracy: 0.9298245614035088
Precision: 0.9363636363636364
Recall: 0.9537037037037037
